In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import time
from datetime import datetime
from scipy import integrate, optimize
import warnings
warnings.filterwarnings('ignore')

# ML libraries
import lightgbm as lgb
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, RidgeCV
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
counntries_df = pd.read_csv("/kaggle/input/countries-data/enriched_covid_19.csv")

In [ ]:
train_df = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/train.csv")
test_df = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/test.csv")
submission = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/submission.csv")

In [ ]:
train_df['Date'].max()

In [ ]:
train_df.describe()
#train_df.info()

In [ ]:
train_df['ConfirmedCases']=train_df['ConfirmedCases'].replace([-1], 0)
train_df['Fatalities']=train_df['Fatalities'].replace([-1], 0)

In [ ]:
train_df[train_df['ConfirmedCases']==-1]

In [ ]:
train_df["key"]=train_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)
test_df["key"]=test_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)

#test_new=pd.merge(test,train, how="left", left_on=["key","Date"], right_on=["key","Date"] )
#train.to_csv(directory + "transfomed.csv")

target1="ConfirmedCases"
target2="Fatalities"
key="key"
def rate(frame, key, target, new_target_name="rate"):
   
    corrections = 0
    group_keys = frame[ key].values.tolist()
    target = frame[target].values.tolist()
    rate=[1.0 for k in range (len(target))]

    for i in range(1, len(group_keys) - 1):
        previous_group = group_keys[i - 1]
        current_group = group_keys[i]

        previous_value = target[i - 1]
        current_value = target[i]
         
        if current_group == previous_group:
                if previous_value!=0.0:
                     rate[i]=current_value/previous_value

                 
        rate[i] =max(1,rate[i] )#correct negative values

    frame[new_target_name] = np.array(rate)

In [ ]:
all_data_train_test=train_df

In [ ]:
all_data_train_test['mean_rate_case_last7']=1.00000000
all_data_train_test['mean_rate_case_last3']=1.00000000
all_data_train_test['mean_rate_case_each7']=1.00000000
all_data_train_test['mean_rate_fat_last7']=1.00000000
all_data_train_test['mean_rate_fat_last3']=1.00000000
all_data_train_test['mean_rate_fat_each7']=1.00000000
all_data_train_test['max_rate_case']=1.00000000
all_data_train_test['min_rate_case']=1.00000000
all_data_train_test['std_rate_case']=1.00000000
all_data_train_test['mode_rate_case']=1.00000000
all_data_train_test['range_rate_case']=1.00000000
all_data_train_test['max_to_min_rate_case']=1.00000000
all_data_train_test['max_rate_fat']=1.00000000
all_data_train_test['min_rate_fat']=1.00000000
all_data_train_test['std_rate_fat']=1.00000000
all_data_train_test['mode_rate_fat']=1.00000000
all_data_train_test['mean_rate_case']=1.00000000
all_data_train_test['mean_rate_fat']=1.00000000
all_data_train_test['range_rate_fat']=1.00000000       
all_data_train_test['max_to_min_rate_fat']=1.00000000   
all_data_train_test['cases_prev']=0  
all_data_train_test['fat_prev']=0   

In [ ]:
rate(all_data_train_test, key, target1, new_target_name="rate_" +target1)
rate(all_data_train_test, key, target2, new_target_name="rate_" +target2)
all_data_train_test.head()
all_data_train_test.info()

In [ ]:
def mean_rate_each(df, country, col,tar):
    target1=df[col][df['key']==country].values.tolist()
    mean_rate_case_each7=[1.0 for k in range (len(target1))]
    j=6
    for i in range(1, len(target1) - 1):
        if (i+j)<=(len(target1)):
            
            current_values1=[target1[i+j - 1],target1[i+j-2],target1[i+j -3],target1[i+j -4],target1[i+j -5],target1[i+j -6],target1[i+j -7]]
           
            mean_rate_case_each7[i+j-1]=np.mean(current_values1)
        else:
            break
                                         
    df[tar][df['key']==country]=np.array(mean_rate_case_each7)
            
    
   


In [ ]:
group_keys = all_data_train_test['key'].values.tolist()
for coc in group_keys:
    mean_rate_each(all_data_train_test, coc, col='rate_ConfirmedCases',tar='mean_rate_case_each7')
    mean_rate_each(all_data_train_test, coc, col='rate_Fatalities',tar='mean_rate_fat_each7')

In [ ]:
all_data_train_test['Province_State'].fillna('NONE', inplace=True)
all_data_train_test['ConfirmedCases'].fillna(0, inplace=True)
all_data_train_test['Fatalities'].fillna(0, inplace=True)
all_data_train_test['Id'].fillna(-1, inplace=True)
#all_data_train_test['ForecastId'].fillna(-1, inplace=True)

In [ ]:
def infection_from_first_confirmed_cases(train_df,country):
    
    confirmed_cases = train_df[(train_df['Country_Region']==country_dict[country]) & train_df['ConfirmedCases']!=0].groupby(['Date']).agg({'ConfirmedCases':['sum']})
    fatalities_cases = train_df[(train_df['Country_Region']==country_dict[country]) & train_df['ConfirmedCases']!=0].groupby(['Date']).agg({'Fatalities':['sum']})
    total_cases = confirmed_cases.join(fatalities_cases)
    country_cases = [i for i in total_cases.ConfirmedCases['sum'].values]
    country_cases_filter = country_cases[0:70] 
       
    return country_cases_filter 

In [ ]:
#get the mean foe last seven vlaues for fatalities and confirmed cases
group_keys = all_data_train_test['key'].values.tolist()
for k in group_keys:
    all_data_train_test['mean_rate_case_last7'][all_data_train_test['key']==k]=all_data_train_test['rate_ConfirmedCases'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)].tail(7).mean()
    all_data_train_test['mean_rate_fat_last7'][all_data_train_test['key']==k]=all_data_train_test['rate_Fatalities'][(all_data_train_test['key']==k)&(all_data_train_test['Fatalities']!=0)].tail(7).mean()
    all_data_train_test['mean_rate_case_last3'][all_data_train_test['key']==k]=all_data_train_test['rate_ConfirmedCases'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)].tail(3).mean()
    all_data_train_test['mean_rate_fat_last3'][all_data_train_test['key']==k]=all_data_train_test['rate_Fatalities'][(all_data_train_test['key']==k)&(all_data_train_test['Fatalities']!=0)].tail(3).mean()
    all_data_train_test['std_rate_case'][all_data_train_test['key']==k]=all_data_train_test['rate_ConfirmedCases'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)].std()
    all_data_train_test['max_rate_case'][all_data_train_test['key']==k]=all_data_train_test['rate_ConfirmedCases'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)].max()
    all_data_train_test['min_rate_case'][all_data_train_test['key']==k]=all_data_train_test['rate_ConfirmedCases'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)].min()
    all_data_train_test['mode_rate_case'][all_data_train_test['key']==k]=all_data_train_test['rate_ConfirmedCases'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)].mode()
    all_data_train_test['mean_rate_case'][all_data_train_test['key']==k]=all_data_train_test['rate_ConfirmedCases'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)].mean()
    all_data_train_test['std_rate_fat'][all_data_train_test['key']==k]=all_data_train_test['rate_Fatalities'][(all_data_train_test['key']==k) & (all_data_train_test['Fatalities']!=0)].std()
    all_data_train_test['max_rate_fat'][all_data_train_test['key']==k]=all_data_train_test['rate_Fatalities'][(all_data_train_test['key']==k) & (all_data_train_test['Fatalities']!=0)].max()
    all_data_train_test['min_rate_fat'][all_data_train_test['key']==k]=all_data_train_test['rate_Fatalities'][(all_data_train_test['key']==k) & (all_data_train_test['Fatalities']!=0)].min()
    all_data_train_test['mode_rate_fat'][all_data_train_test['key']==k]=all_data_train_test['rate_Fatalities'][(all_data_train_test['key']==k) & (all_data_train_test['Fatalities']!=0)].mode()
    all_data_train_test['mean_rate_fat'][all_data_train_test['key']==k]=all_data_train_test['rate_Fatalities'][(all_data_train_test['key']==k) & (all_data_train_test['Fatalities']!=0)].mean()
   # all_data_train_test['max_to_min_rate_case'][all_data_train_test['key']==k]=(all_data_train_test['max_rate_case'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)])/(all_data_train_test['min_rate_case'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)])
    #all_data_train_test['max_to_min_rate_fat'][all_data_train_test['key']==k]=(all_data_train_test['max_rate_fat'][(all_data_train_test['key']==k) & (all_data_train_test['Fatalities']!=0)])/(all_data_train_test['min_rate_fat'][(all_data_train_test['key']==k) & (all_data_train_test['Fatalities']!=0)])
    
#all_data_train_test['cases_prev']=all_data_train_test['ConfirmedCases'].shift()
#all_data_train_test['fat_prev']=all_data_train_test['Fatalities'].shift()                   

all_data_train_test['max_to_min_rate_case']=all_data_train_test['max_rate_case']/all_data_train_test['min_rate_case']
all_data_train_test['max_to_min_rate_fat']=all_data_train_test['max_rate_fat']/all_data_train_test['min_rate_fat']

In [ ]:
all_data_train_test['mean_rate_case_last7']=(all_data_train_test[(all_data_train_test['rate_ConfirmedCases']!=0) & (all_data_train_test['ConfirmedCases']!=0)].groupby(['key'])).tail(7).mean()

In [ ]:
all_data_train_test['ConfirmedCases'][all_data_train_test['ConfirmedCases']==-1]

In [ ]:
train_df['Date'].max()

In [ ]:
all_data_train_test['cases_prev'] = all_data_train_test.groupby("key")["ConfirmedCases"].shift()
all_data_train_test['fat_prev'] = all_data_train_test.groupby("key")["Fatalities"].shift()

In [ ]:
for lag in range(1, ((train_df['Date'].max()-train_df['Date'].min()).days)):            
    all_data_train_test[f"lag_{lag}_cc"] = all_data_train_test.groupby("key")["ConfirmedCases"].shift(lag)
    all_data_train_test[f"lag_{lag}_ft"] = all_data_train_test.groupby("key")["Fatalities"].shift(lag)
 

In [ ]:
all_data_train_test[['mean_rate_case_last7','ConfirmedCases','rate_ConfirmedCases']][all_data_train_test['Country_Region']=='Qatar']

In [ ]:

all_data_train_test[all_data_train_test['Country_Region']=='Qatar'].tail(50)

In [ ]:
counntries_df.rename(columns={'Country/Region':'Country_Region','Province/State':'Province_State'},inplace = True)
counntries_df["key"]=counntries_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)


In [ ]:
counntries_df.columns

In [ ]:
contries_data_columns=['Province_State', 'Country_Region', 'Lat', 'Long', 'Date',
       'ConfirmedCases', 'Fatalities', 'age_0-4', 'age_5-9', 'age_10-14','key',
       'age_15-19', 'age_20-24', 'age_25-29', 'age_30-34', 'age_35-39',
       'age_40-44', 'age_45-49', 'age_50-54', 'age_55-59', 'age_60-64',
       'age_65-69', 'age_70-74', 'age_75-79', 'age_80-84', 'age_85-89',
       'age_90-94', 'age_95-99', 'age_100+', 'total_pop', 'smokers_perc',
       'density', 'urbanpop', 'hospibed', 'lung', 'femalelung', 'malelung',
       'restrictions', 'quarantine', 'schools']

In [ ]:
grouped_countries=counntries_df.groupby(['Country_Region'], as_index=False).agg({"Lat": "max","Long": "max",'total_pop': "max",
                                                                                         'smokers_perc': "max",'density': "max",'urbanpop': "max",
                                                                                         'hospibed': "max",'lung': "max",'femalelung': "max",
                                                                                         'malelung': "max",'restrictions': "max",'quarantine': "max",
                                                                                         'schools': "max",'age_0-4': "max",'age_100+': "max",'age_5-9': "max",
                                                                                         'age_95-99': "max",'age_90-94': "max",'age_85-89': "max",
                                                                                         'age_80-84': "max",'age_75-79': "max",'age_70-74': "max",
                                                                                         'age_65-69': "max",'age_60-64': "max",'age_55-59': "max",
                                                                                         'age_50-54': "max",'age_45-49': "max",'age_40-44': "max",
                                                                                         'age_35-39': "max",'age_30-34': "max",'age_25-29': "max",
                                                                                         'age_20-24': "max",'age_15-19': "max",'age_10-14': "max",
                                                                                         })

In [ ]:
grouped_countries=counntries_df.groupby(['key'], as_index=False).agg({"Lat": "max","Long": "max",'total_pop': "max",
                                                                                         'smokers_perc': "max",'density': "max",'urbanpop': "max",
                                                                                         'hospibed': "max",'lung': "max",'femalelung': "max",
                                                                                         'malelung': "max",'restrictions': "max",'quarantine': "max",
                                                                                         'schools': "max",'age_0-4': "max",'age_100+': "max",'age_5-9': "max",
                                                                                         'age_95-99': "max",'age_90-94': "max",'age_85-89': "max",
                                                                                         'age_80-84': "max",'age_75-79': "max",'age_70-74': "max",
                                                                                         'age_65-69': "max",'age_60-64': "max",'age_55-59': "max",
                                                                                         'age_50-54': "max",'age_45-49': "max",'age_40-44': "max",
                                                                                         'age_35-39': "max",'age_30-34': "max",'age_25-29': "max",
                                                                                         'age_20-24': "max",'age_15-19': "max",'age_10-14': "max",
                                                                                         })

In [ ]:
merged_train_df = pd.merge(all_data_train_test,
                 grouped_countries[['Lat', 'Long',
        'age_0-4', 'age_5-9', 'age_10-14','key',
       'age_15-19', 'age_20-24', 'age_25-29', 'age_30-34', 'age_35-39',
       'age_40-44', 'age_45-49', 'age_50-54', 'age_55-59', 'age_60-64',
       'age_65-69', 'age_70-74', 'age_75-79', 'age_80-84', 'age_85-89',
       'age_90-94', 'age_95-99', 'age_100+', 'total_pop', 'smokers_perc',
       'density', 'urbanpop', 'hospibed', 'lung', 'femalelung', 'malelung',
       'restrictions', 'quarantine', 'schools']],
                 on='key',how='left')

In [ ]:
merged_train_df['Date']= pd.to_datetime(merged_train_df['Date']) 

In [ ]:
merged_train_df['confirmed_to_pop']=merged_train_df['ConfirmedCases']/(merged_train_df['total_pop']*1000)
merged_train_df['fat_to_pop']=merged_train_df['Fatalities']/(merged_train_df['total_pop']*1000)

In [ ]:
merged_train_df[['confirmed_to_pop','ConfirmedCases','total_pop']][merged_train_df['Country_Region']=='Saudi Arabia']

In [ ]:
merged_train_df.info()

In [ ]:
merged_train_df.columns

In [ ]:
merged_train_df[merged_train_df.isnull().any(axis=1)]

In [ ]:
merged_train_df['Province_State'].fillna('NONE', inplace=True)

In [ ]:
merged_train_df.fillna(0, inplace=True)

In [ ]:
merged_train_df.head()

In [ ]:
number_c = merged_train_df['Country_Region']
countries = (merged_train_df['Country_Region'])
country_dict = dict(zip(countries, number_c)) 
country_dict

In [ ]:
merged_train_df.columns

In [ ]:
merged_train_df['Date_num'] = pd.to_datetime(merged_train_df['Date'])
merged_train_df['Date_num'] = merged_train_df['Date_num'].dt.strftime("%m%d")
merged_train_df['Date_num']

In [ ]:
merged_test_df['Date_num'] = pd.to_datetime(merged_test_df['Date'])
merged_test_df['Date_num'] = merged_test_df['Date_num'].dt.strftime("%m%d")
merged_test_df['Date_num']

In [ ]:
merged_train_df['Date_num']=merged_train_df['Date_num'].astype('int')
merged_test_df['Date_num']=merged_test_df['Date_num'].astype('int')
merged_train_df.info()

## Combined

In [ ]:
merged_train_df.columns

In [ ]:
#'rate_ConfirmedCases','rate_Fatalities',

from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
merged_train_df[['mean_rate_case_last7', 'mean_rate_case_each7','mean_rate_case_last3','mean_rate_fat_last3',
       'mean_rate_fat_last7', 'mean_rate_fat_each7', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat','cases_prev', 'fat_prev',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat',
       'max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities','total_pop', 'smokers_perc', 'density']] = mms.fit_transform(merged_train_df[['mean_rate_case_last7', 'mean_rate_case_each7',
       'mean_rate_fat_last7', 'mean_rate_fat_each7', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case','cases_prev', 'fat_prev',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat','mean_rate_case_last3','mean_rate_fat_last3',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat',
       'max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities','total_pop', 'smokers_perc', 'density']])

In [ ]:
train = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/train.csv')

In [ ]:
train.info()

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder

import xgboost as xgb

from tensorflow.keras.optimizers import Nadam
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import tensorflow.keras.layers as KL
from datetime import timedelta
import numpy as np
import pandas as pd


import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge

import datetime
import gc
from tqdm import tqdm
def get_cpmp_sub(save_oof=False, save_public_test=False):
    train = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/train.csv')
    train['Province_State'].fillna('', inplace=True)
    train['Date'] = pd.to_datetime(train['Date'])
    train['day'] = train.Date.dt.dayofyear
    #train = train[train.day <= 85]
    train['geo'] = ['_'.join(x) for x in zip(train['Country_Region'], train['Province_State'])]
    train.fillna(0, inplace=True)
    train['ConfirmedCases']=train['ConfirmedCases'].replace([-1], 0)
    train['Fatalities']=train['Fatalities'].replace([-1], 0)
    train

    test = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/test.csv')
    test['Province_State'].fillna('', inplace=True)
    test['Date'] = pd.to_datetime(test['Date'])
    test['day'] = test.Date.dt.dayofyear
    test['geo'] = ['_'.join(x) for x in zip(test['Country_Region'], test['Province_State'])]
    test.fillna(0, inplace=True)
    test

    day_min = train['day'].min()
    train['day'] -= day_min
    test['day'] -= day_min

    min_test_val_day = test.day.min()
    max_test_val_day = train.day.max()
    max_test_day = test.day.max()
    num_days = max_test_day + 1

    min_test_val_day, max_test_val_day, num_days

    train['ForecastId'] = -1
    test['Id'] = -1
    test['ConfirmedCases'] = 0
    test['Fatalities'] = 0

    debug = False

    data = pd.concat([train,
                      test[test.day > max_test_val_day][train.columns]
                     ]).reset_index(drop=True)
    if debug:
        data = data[data['geo'] >= 'France_'].reset_index(drop=True)
    #del train, test
    gc.collect()

    dates = data[data['geo'] == 'France_'].Date.values

    if 0:
        gr = data.groupby('geo')
        data['ConfirmedCases'] = gr.ConfirmedCases.transform('cummax')
        data['Fatalities'] = gr.Fatalities.transform('cummax')

    geo_data = data.pivot(index='geo', columns='day', values='ForecastId')
    num_geo = geo_data.shape[0]
    geo_data

    geo_id = {}
    for i,g in enumerate(geo_data.index):
        geo_id[g] = i


    ConfirmedCases = data.pivot(index='geo', columns='day', values='ConfirmedCases')
    Fatalities = data.pivot(index='geo', columns='day', values='Fatalities')

    if debug:
        cases = ConfirmedCases.values
        deaths = Fatalities.values
    else:
        cases = np.log1p(ConfirmedCases.values)
        deaths = np.log1p(Fatalities.values)


    def get_dataset(start_pred, num_train, lag_period): #63,5,14
        days = np.arange( start_pred - num_train + 1, start_pred + 1)#(59,1,64)
        lag_cases = np.vstack([cases[:, d - lag_period : d] for d in days])
        lag_deaths = np.vstack([deaths[:, d - lag_period : d] for d in days])
        target_cases = np.vstack([cases[:, d : d + 1] for d in days])
        target_deaths = np.vstack([deaths[:, d : d + 1] for d in days])
        geo_ids = np.vstack([geo_ids_base for d in days])
        country_ids = np.vstack([country_ids_base for d in days])
        return lag_cases, lag_deaths, target_cases, target_deaths, geo_ids, country_ids, days

    def update_valid_dataset(data, pred_death, pred_case):
        lag_cases, lag_deaths, target_cases, target_deaths, geo_ids, country_ids, days = data
        day = days[-1] + 1
        new_lag_cases = np.hstack([lag_cases[:, 1:], pred_case])
        new_lag_deaths = np.hstack([lag_deaths[:, 1:], pred_death]) 
        new_target_cases = cases[:, day:day+1]
        new_target_deaths = deaths[:, day:day+1] 
        new_geo_ids = geo_ids  
        new_country_ids = country_ids  
        new_days = 1 + days
        return new_lag_cases, new_lag_deaths, new_target_cases, new_target_deaths, new_geo_ids, new_country_ids, new_days

    def fit_eval(lr_death, lr_case, data, start_lag_death, end_lag_death, num_lag_case, fit, score):
        lag_cases, lag_deaths, target_cases, target_deaths, geo_ids, country_ids, days = data

        X_death = np.hstack([lag_cases[:, -start_lag_death:-end_lag_death], country_ids])
        X_death = np.hstack([lag_deaths[:, -num_lag_case:], country_ids])
        X_death = np.hstack([lag_cases[:, -start_lag_death:-end_lag_death], lag_deaths[:, -num_lag_case:], country_ids])
        y_death = target_deaths
        y_death_prev = lag_deaths[:, -1:]
        if fit:
            if 0:
                keep = (y_death > 0).ravel()
                X_death = X_death[keep]
                y_death = y_death[keep]
                y_death_prev = y_death_prev[keep]
            lr_death.fit(X_death, y_death)
        y_pred_death = lr_death.predict(X_death)
        y_pred_death = np.maximum(y_pred_death, y_death_prev)

        X_case = np.hstack([lag_cases[:, -num_lag_case:], geo_ids])
        X_case = lag_cases[:, -num_lag_case:]
        y_case = target_cases
        y_case_prev = lag_cases[:, -1:]
        if fit:
            lr_case.fit(X_case, y_case)
        y_pred_case = lr_case.predict(X_case)
        y_pred_case = np.maximum(y_pred_case, y_case_prev)

        if score:
            death_score = val_score(y_death, y_pred_death)
            case_score = val_score(y_case, y_pred_case)
        else:
            death_score = 0
            case_score = 0

        return death_score, case_score, y_pred_death, y_pred_case

    def train_model(train, valid, start_lag_death, end_lag_death, num_lag_case, num_val, score=True):
        alpha = 3
        lr_death = Ridge(alpha=alpha, fit_intercept=False)
        lr_case = Ridge(alpha=alpha, fit_intercept=True)

        (train_death_score, train_case_score, train_pred_death, train_pred_case,
        ) = fit_eval(lr_death, lr_case, train, start_lag_death, end_lag_death, num_lag_case, fit=True, score=score)

        death_scores = []
        case_scores = []

        death_pred = []
        case_pred = []

        for i in range(num_val):

            (valid_death_score, valid_case_score, valid_pred_death, valid_pred_case,
            ) = fit_eval(lr_death, lr_case, valid, start_lag_death, end_lag_death, num_lag_case, fit=False, score=score)

            death_scores.append(valid_death_score)
            case_scores.append(valid_case_score)
            death_pred.append(valid_pred_death)
            case_pred.append(valid_pred_case)

            if 0:
                print('val death: %0.3f' %  valid_death_score,
                      'val case: %0.3f' %  valid_case_score,
                      'val : %0.3f' %  np.mean([valid_death_score, valid_case_score]),
                      flush=True)
            valid = update_valid_dataset(valid, valid_pred_death, valid_pred_case)

        if score:
            death_scores = np.sqrt(np.mean([s**2 for s in death_scores]))
            case_scores = np.sqrt(np.mean([s**2 for s in case_scores]))
            if 0:
                print('train death: %0.3f' %  train_death_score,
                      'train case: %0.3f' %  train_case_score,
                      'val death: %0.3f' %  death_scores,
                      'val case: %0.3f' %  case_scores,
                      'val : %0.3f' % ( (death_scores + case_scores) / 2),
                      flush=True)
            else:
                print('%0.4f' %  case_scores,
                      ', %0.4f' %  death_scores,
                      '= %0.4f' % ( (death_scores + case_scores) / 2),
                      flush=True)
        death_pred = np.hstack(death_pred)
        case_pred = np.hstack(case_pred)
        return death_scores, case_scores, death_pred, case_pred

    countries = [g.split('_')[0] for g in geo_data.index]
    countries = pd.factorize(countries)[0]

    country_ids_base = countries.reshape((-1, 1))
    ohe = OneHotEncoder(sparse=False)
    country_ids_base = 0.2 * ohe.fit_transform(country_ids_base)
    country_ids_base.shape

    geo_ids_base = np.arange(num_geo).reshape((-1, 1))
    ohe = OneHotEncoder(sparse=False)
    geo_ids_base = 0.1 * ohe.fit_transform(geo_ids_base)
    geo_ids_base.shape

    def val_score(true, pred):
        pred = np.log1p(np.round(np.expm1(pred) - 0.2))
        return np.sqrt(mean_squared_error(true.ravel(), pred.ravel()))

    def val_score(true, pred):
        return np.sqrt(mean_squared_error(true.ravel(), pred.ravel()))

#14,6,5,14

    start_lag_death, end_lag_death = 14, 6,
    num_train = 7
    num_lag_case = 14
    lag_period = max(start_lag_death, num_lag_case)  #14

    def get_oof(start_val_delta=0):   
        start_val = min_test_val_day + start_val_delta
        last_train = start_val - 1
        num_val = max_test_val_day - start_val + 1
        print(dates[start_val], start_val, num_val)
        train_data = get_dataset(last_train, num_train, lag_period)
        valid_data = get_dataset(start_val, 1, lag_period)
        _, _, val_death_preds, val_case_preds = train_model(train_data, valid_data, 
                                                            start_lag_death, end_lag_death, num_lag_case, num_val)

        pred_deaths = Fatalities.iloc[:, start_val:start_val+num_val].copy()
        pred_deaths.iloc[:, :] = np.expm1(val_death_preds)
        pred_deaths = pred_deaths.stack().reset_index()
        pred_deaths.columns = ['geo', 'day', 'Fatalities']
        pred_deaths

        pred_cases = ConfirmedCases.iloc[:, start_val:start_val+num_val].copy()
        pred_cases.iloc[:, :] = np.expm1(val_case_preds)
        pred_cases = pred_cases.stack().reset_index()
        pred_cases.columns = ['geo', 'day', 'ConfirmedCases']
        pred_cases

        sub = train[['Date', 'Id', 'geo', 'day']]
        sub = sub.merge(pred_cases, how='left', on=['geo', 'day'])
        sub = sub.merge(pred_deaths, how='left', on=['geo', 'day'])
        #sub = sub.fillna(0)
        sub = sub[sub.day >= start_val]
        sub = sub[['Id', 'ConfirmedCases', 'Fatalities']].copy()
        return sub

#'2020-03-27', '2020-03-24', '2020-03-21', '2020-03-18'
#'2020-03-22', '2020-03-19', '2020-03-16', '2020-03-13'
    if save_oof:
        for start_val_delta, date in zip(range(3, -8, -3),
                                  ['2020-03-22', '2020-03-19', '2020-03-16', '2020-03-13']):
            print(date, end=' ')
            oof = get_oof(start_val_delta)
            oof.to_csv('../submissions/cpmp-%s.csv' % date, index=None)

    def get_sub(start_val_delta=0):   
        start_val = min_test_val_day + start_val_delta  #64
        last_train = start_val - 1   #63
        num_val = max_test_val_day - start_val + 1 #11
        print(dates[last_train], start_val, num_val)
        num_lag_case = 14  #14
        train_data = get_dataset(last_train, num_train, lag_period)#63,5,14
        valid_data = get_dataset(start_val, 1, lag_period) #64,1,14
        _, _, val_death_preds, val_case_preds = train_model(train_data, valid_data, 
                                                            start_lag_death, end_lag_death, num_lag_case, num_val)

        pred_deaths = Fatalities.iloc[:, start_val:start_val+num_val].copy()
        pred_deaths.iloc[:, :] = np.expm1(val_death_preds)
        pred_deaths = pred_deaths.stack().reset_index()
        pred_deaths.columns = ['geo', 'day', 'Fatalities']
        pred_deaths

        pred_cases = ConfirmedCases.iloc[:, start_val:start_val+num_val].copy()
        pred_cases.iloc[:, :] = np.expm1(val_case_preds)
        pred_cases = pred_cases.stack().reset_index()
        pred_cases.columns = ['geo', 'day', 'ConfirmedCases']
        pred_cases

        sub = test[['Date', 'ForecastId', 'geo', 'day']]
        sub = sub.merge(pred_cases, how='left', on=['geo', 'day'])
        sub = sub.merge(pred_deaths, how='left', on=['geo', 'day'])
        sub = sub.fillna(0)
        sub = sub[['ForecastId', 'ConfirmedCases', 'Fatalities']]
        return sub
        return sub


    known_test = train[['geo', 'day', 'ConfirmedCases', 'Fatalities']
              ].merge(test[['geo', 'day', 'ForecastId']], how='left', on=['geo', 'day'])
    known_test = known_test[['ForecastId', 'ConfirmedCases', 'Fatalities']][known_test.ForecastId.notnull()].copy()
    known_test

    unknow_test = test[test.day > max_test_val_day]
    unknow_test

    def get_final_sub():   
        start_val = max_test_val_day + 1
        last_train = start_val - 1
        num_val = max_test_day - start_val + 1
        print(dates[last_train], start_val, num_val)
        num_lag_case = num_val + 3
        train_data = get_dataset(last_train, num_train, lag_period)
        valid_data = get_dataset(start_val, 1, lag_period)
        (_, _, val_death_preds, val_case_preds
        ) = train_model(train_data, valid_data, start_lag_death, end_lag_death, num_lag_case, num_val, score=False)

        pred_deaths = Fatalities.iloc[:, start_val:start_val+num_val].copy()
        pred_deaths.iloc[:, :] = np.expm1(val_death_preds)
        pred_deaths = pred_deaths.stack().reset_index()
        pred_deaths.columns = ['geo', 'day', 'Fatalities']
        pred_deaths

        pred_cases = ConfirmedCases.iloc[:, start_val:start_val+num_val].copy()
        pred_cases.iloc[:, :] = np.expm1(val_case_preds)
        pred_cases = pred_cases.stack().reset_index()
        pred_cases.columns = ['geo', 'day', 'ConfirmedCases']
        pred_cases
        print(unknow_test.shape, pred_deaths.shape, pred_cases.shape)

        sub = unknow_test[['Date', 'ForecastId', 'geo', 'day']]
        sub = sub.merge(pred_cases, how='left', on=['geo', 'day'])
        sub = sub.merge(pred_deaths, how='left', on=['geo', 'day'])
        #sub = sub.fillna(0)
        sub = sub[['ForecastId', 'ConfirmedCases', 'Fatalities']]
        sub = pd.concat([known_test, sub])
        return sub

    if save_public_test:
        sub = get_sub()
    else:
        sub = get_final_sub()
    return sub

In [ ]:
merged_train_df.head()


# LGB

In [ ]:
from datetime import datetime as dt, timedelta
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
train_df = pd.read_csv("../input/covid19-global-forecasting-week-4/train.csv")

train_df['cases_prev']=train_df['ConfirmedCases'].shift()
train_df['fat_prev']=train_df['Fatalities'].shift() 
train_df["key"]=train_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)

sub_df = pd.read_csv("../input/covid19-global-forecasting-week-4/test.csv")

sub_df["key"]=sub_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)

'''coo_d = merged_train_df.groupby("Country_Region")[['mean_rate_case_last7', 'mean_rate_case_each7',
       'mean_rate_fat_last7', 'mean_rate_fat_each7', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case','mean_rate_case_last3','mean_rate_fat_last3',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat','Lat','Long',
       'max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities','total_pop', 'smokers_perc', 'density']].mean().reset_index()'''

coo_d = merged_train_df.groupby('key')[['mean_rate_case_last7', 'mean_rate_case_each7',
       'mean_rate_fat_last7', 'mean_rate_fat_each7', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case','mean_rate_case_last3','mean_rate_fat_last3',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat','Lat','Long',
       'max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities','total_pop', 'smokers_perc', 'density']].mean().reset_index()

loc_group = ['key']
def preprocess(df):
        df["Date"] = df["Date"].astype("datetime64[ms]")
     #   df["days"] = (df["Date"] - pd.to_datetime("2020-01-01")).dt.days
    #    df["weekend"] = df["Date"].dt.dayofweek//5

        df = df.merge(coo_d, how="left", on='key')
        df["Lat"] = (df["Lat"] // 30).astype(np.float32).fillna(0)
        df["Long"] = (df["Long"] // 60).astype(np.float32).fillna(0)

        for col in loc_group:
            df[col].fillna("none", inplace=True)
        return df
features2=['Id', 'Province_State', 'Country_Region', 'Date','key',
       'mean_rate_case_last7', 'mean_rate_case_each7', 'mean_rate_fat_last7',
       'mean_rate_fat_each7', 'max_rate_case', 'min_rate_case','mean_rate_case_last3','mean_rate_fat_last3',
       'std_rate_case', 'mode_rate_case', 'range_rate_case','ConfirmedCases',
       'Fatalities','cases_prev','fat_prev','max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat',
       'Lat', 'Long', 'max_to_min_rate_fat', 'rate_ConfirmedCases',
       'rate_Fatalities', 'total_pop', 'smokers_perc', 'density']
merged_train_df2 = pd.DataFrame(columns=features2)

merged_train_df2=merged_train_df[features2]
sub_df = preprocess(sub_df)
merged_train_df2['Date']= pd.to_datetime(merged_train_df2['Date']) 
#x_train = merged_train_df2
x_train = preprocess(train_df)
x_test = sub_df
x_train.loc[x_train["Date"]<x_test['Date'].min(),"split"] = "train"
x_train.loc[x_train["Date"]>=x_test['Date'].min(),"split"] = "test"

In [ ]:
sub_df.shape,x_train.shape

In [ ]:
merged_train_df2.columns[~merged_train_df2.columns.isin(sub_df.columns)]

In [ ]:
x_train

In [ ]:
train_df['Date'].max()

In [ ]:
x_test['Date'].min()

In [ ]:
df.columns

In [ ]:
features = ['mean_rate_case_last7', 'mean_rate_case_each7',
       'mean_rate_fat_last7', 'mean_rate_fat_each7', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat','mean_rate_case_last3','mean_rate_fat_last3',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat','Lat','Long',
       'max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities','total_pop', 'smokers_perc', 'density']
def train_model(df, label, base_label, features=features, **kwargs):
    X_train = df.loc[df["split"] == "train"][features]
    y_train = np.log(df.loc[df["split"] == "train"][label] + 1)
    b_train = np.log(df.loc[df["split"] == "train"][base_label] + 1)
    X_test = df.loc[df["split"] == "test"][features]
    y_test = np.log(df.loc[df["split"] == "test", label] + 1)
    b_test = np.log(df.loc[df["split"] == "test", base_label] + 1)
    print(kwargs)
    model = lgb.LGBMRegressor(**kwargs)
    model.fit(X_train, y_train, init_score = b_train)
    y_pred = model.predict(X_test)
    print(np.sqrt(mean_squared_error(y_test, y_pred + b_test)))
   # print(len(X_test))
    return model

In [ ]:
lgb_model_cases = train_model(x_train,"ConfirmedCases",'cases_prev',features,
                                   max_depth=5,
                                   colsample_bytree=0.8,
                                   learning_rate=0.1,
                                   n_estimators=1000,
                                   subsample=0.8)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(lgb_model_cases.feature_importances_,x_train[features])), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-01.png')

In [ ]:
lgb_model_fatalities = train_model(x_train, "Fatalities",'fat_prev',features, 
                                   max_depth=5,
                                   colsample_bytree=0.8,
                                   learning_rate=0.1,
                                   n_estimators=500,
                                   subsample=0.8
                                  )

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(lgb_model_fatalities.feature_importances_,x_train[features])), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-01.png')

In [ ]:
X_train = x_train[features]
y_train = np.log(x_train["ConfirmedCases"] + 1)
b_train = np.log(x_train["cases_prev"] + 1)
cases_model = lgb.LGBMRegressor(max_depth=5,
                                   colsample_bytree=0.8,
                                   learning_rate=0.1,
                                   n_estimators=500,
                                   subsample=0.8
                               )
cases_model.fit(X_train, y_train, init_score = b_train)

X_train = x_train[features]
y_train = np.log(x_train["Fatalities"] + 1)
b_train = np.log(x_train["fat_prev"] + 1)
fatalities_model = lgb.LGBMRegressor(max_depth=5,
                                   colsample_bytree=0.8,
                                   learning_rate=0.1,
                                   n_estimators=500,
                                   subsample=0.8)
fatalities_model.fit(X_train, y_train, init_score = b_train)

In [ ]:
sub_df.shape,merged_train_df2.shape

In [ ]:
merged_train_df2.columns

In [ ]:
merged_train_df2.info()

In [ ]:
train_df['Date']

In [ ]:
merged_train_df.columns

In [ ]:
df = pd.read_csv("../input/covid19-global-forecasting-week-4/train.csv")
sub_df = pd.read_csv("../input/covid19-global-forecasting-week-4/test.csv")
df["key"]=train_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)
sub_df["key"]=sub_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)
features_columns = ['Id', 'Province_State', 'Country_Region', 'Date', 'ConfirmedCases',
       'Fatalities','mean_rate_case_last7', 'mean_rate_case_each7',
       'mean_rate_fat_last7', 'mean_rate_fat_each7', 'max_rate_case','mean_rate_case_last3','mean_rate_fat_last3',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat','Lat','Long',
       'max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities','total_pop', 'smokers_perc', 'density']

coo_df = merged_train_df.groupby("key")[['mean_rate_case_last7', 'mean_rate_case_each7',
       'mean_rate_fat_last7', 'mean_rate_fat_each7', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat','mean_rate_case_last3','mean_rate_fat_last3',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat','Lat','Long',
       'max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities','total_pop', 'smokers_perc', 'density']].mean().reset_index()
   # coo_df = coo_df[coo_df["Country_Region"].notnull()]

loc_group = ["key"]
   # merged_train_df['Date']= pd.to_datetime(merged_train_df['Date']) 
  #  merged_train_df["days"] = (merged_train_df["Date"] - pd.to_datetime("2020-01-01")).dt.days

def preprocess(df):
    df["Date"] = df["Date"].astype("datetime64[ms]")
  #  df["days"] = (df["Date"] - pd.to_datetime("2020-01-01")).dt.days
    #    df["weekend"] = df["Date"].dt.dayofweek//5

    df = df.merge(coo_df, how="left", on="key")
    df["Lat"] = (df["Lat"] // 30).astype(np.float32).fillna(0)
    df["Long"] = (df["Long"] // 60).astype(np.float32).fillna(0)

    for col in loc_group:
        df[col].fillna("none", inplace=True)
    return df
merged_train_df2=pd.DataFrame(columns=features_columns)
merged_train_df2=merged_train_df[features_columns].copy()
    #df = merged_train_df
df = preprocess(df)
sub_df = preprocess(sub_df)

In [ ]:
train_df = pd.read_csv("../input/covid19-global-forecasting-week-4/train.csv")
test_df = pd.read_csv("../input/covid19-global-forecasting-week-4/test.csv")

#train_df['Date']= pd.to_datetime(train_df['Date']) 
country_list = train_df ['Country_Region'].unique()
key_list=df['key'].unique()
#train_df['Date']= pd.to_datetime(train_df['Date']) 
scoring_dates = test_df['Date'].unique()
pred_df = pd.DataFrame(columns=train_df.columns)
features = ['mean_rate_case_last7', 'mean_rate_case_each7',
       'mean_rate_fat_last7', 'mean_rate_fat_each7', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case','mean_rate_case_last3','mean_rate_fat_last3',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat','Lat','Long',
       'max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities','total_pop', 'smokers_perc', 'density']
'''for date in scoring_dates.tolist():
    print(date)
    new_df = base_df.loc[base_df["Date"] < date].copy()
    curr_date_df = test.loc[test["Date"] == date].copy()
    curr_date_df["ConfirmedCases"] = 0
    curr_date_df["Fatalities"] = 0
    new_df = new_df.append(curr_date_df).reset_index(drop=True)
    new_df = generate_features(new_df)
    new_df[features] = new_df[features]
    predictions = cases_model.predict(new_df[features]) + np.log(new_df["ConfirmedCases_yesterday"] + 1)
    new_df["predicted_cases"] = round(np.maximum(np.exp(predictions) - 1, new_df["ConfirmedCases_yesterday"]))
    predictions = fatalities_model.predict(new_df[features]) + np.log(new_df["Fatalities_yesterday"] + 1)
    new_df["predicted_fatalities"] = np.maximum(np.exp(predictions) - 1, new_df["Fatalities_yesterday"])
    new_df["predicted_fatalities"] = round(np.minimum(new_df["predicted_fatalities"], new_df["predicted_cases"]*0.2))
    new_df.loc[new_df["Date"] == date, "ConfirmedCases"] = new_df.loc[new_df["Date"] == date, "predicted_cases"]
    new_df.loc[new_df["Date"] == date, "Fatalities"] = new_df.loc[new_df["Date"] == date, "predicted_fatalities"]
    pred_df = pred_df.append(new_df.loc[new_df["Date"] == date][pred_df.columns.tolist()])
    if date not in base_df["Date"].unique():
        base_df = base_df.append(new_df.loc[new_df["Date"] == date][base_df.columns.tolist()])'''

'''for date in scoring_dates.tolist():
    print(date)
    new_df = merged_train_df2.loc[merged_train_df2["Date"] < date].copy()
    curr_date_df = sub_df.loc[sub_df["Date"] == date].copy()
    curr_date_df["ConfirmedCases"] = 0
    curr_date_df["Fatalities"] = 0
    curr_date_df["cases_prev"] = 0
    curr_date_df["fat_prev"] = 0
    new_df = new_df.append(curr_date_df).reset_index(drop=True)
    #new_df['Date']= pd.to_datetime(new_df['Date']) 
    #new_df = preprocess(new_df)
    #new_df[features] = new_df[features]
 
    predictions = cases_model.predict(new_df[features]) + np.log(new_df["cases_prev"] + 1)
    new_df["predicted_cases"] = round(np.maximum(np.exp(predictions) - 1, new_df["cases_prev"]))
    predictions = fatalities_model.predict(new_df[features]) + np.log(new_df["fat_prev"] + 1)
    new_df["predicted_fatalities"] = np.maximum(np.exp(predictions) - 1, new_df["fat_prev"])
    new_df["predicted_fatalities"] = round(np.minimum(new_df["predicted_fatalities"], new_df["predicted_cases"]*0.2))
    new_df.loc[new_df["Date"] == date, "ConfirmedCases"] = new_df.loc[new_df["Date"] == date, "predicted_cases"]
    new_df.loc[new_df["Date"] == date, "Fatalities"] = new_df.loc[new_df["Date"] == date, "predicted_fatalities"]
    pred_df = pred_df.append(new_df.loc[new_df["Date"] == date][pred_df.columns.tolist()])
    
    if date not in train_df["Date"].unique():
                
        train_df = train_df.append(new_df.loc[new_df["Date"] == date][train_df.columns.tolist()])'''
#####################################################        
'''sub = []
for date in scoring_dates.tolist():
    
    for country in country_list:
        province_list = x_train.loc[x_train['Country_Region'] == country].Province_State.unique()
        for province in province_list:
            new_df = merged_train_df2.loc[merged_train_df2["Date"] < date].copy()
            curr_date_df = sub_df.loc[sub_df["Date"] == date].copy()
            curr_date_df["ConfirmedCases"] = 0
            curr_date_df["Fatalities"] = 0
            new_df = new_df.append(curr_date_df).reset_index(drop=True)
            predictions = cases_model.predict(new_df[features]) + np.log(new_df["cases_prev"] + 1)
            new_df["predicted_cases"] = round(np.maximum(np.exp(predictions) - 1, new_df["cases_prev"]))
            predictions = fatalities_model.predict(new_df[features]) + np.log(new_df["fat_prev"] + 1)
            new_df["predicted_fatalities"] = np.maximum(np.exp(predictions) - 1, new_df["fat_prev"])
            new_df["predicted_fatalities"] = round(np.minimum(new_df["predicted_fatalities"], new_df["predicted_cases"]*0.2))
            new_df.loc[new_df["Date"] == date, "ConfirmedCases"] = new_df.loc[new_df["Date"] == date, "predicted_cases"]
            new_df.loc[new_df["Date"] == date, "Fatalities"] = new_df.loc[new_df["Date"] == date, "predicted_fatalities"]
            pred_df = pred_df.append(new_df.loc[new_df["Date"] == date][pred_df.columns.tolist()])
            
        
            
            X_forecastId = sub_df.loc[(sub_df['Country_Region'] == country) & (sub_df['Province_State'] == province), ['ForecastId']]
            X_forecastId = X_forecastId.values.tolist()
            X_forecastId = [v[0] for v in X_forecastId]
          
for j in range(len(sub_df['ForecastId'])):
       
     dic = { 'ForecastId': X_forecastId[j], 'ConfirmedCases': pred_df['ConfirmedCases'][j], 'Fatalities': pred_df['Fatalities'][j]}
     sub.append(dic)

# %% [code]
submission = pd.DataFrame(sub)
submission[['ForecastId','ConfirmedCases','Fatalities']].to_csv(path_or_buf='submission.csv',index=False)   '''
######################################################################################################################################
'''sub = []
for date in scoring_dates.tolist():
    print(date)
    
    for key in key_list:
              
        new_df = merged_train_df2.loc[merged_train_df2["Date"] < date].copy()
        curr_date_df = sub_df.loc[sub_df["Date"] == date].copy()
        curr_date_df["ConfirmedCases"] = 0
        curr_date_df["Fatalities"] = 0
        new_df = new_df.append(curr_date_df).reset_index(drop=True)
        predictions = cases_model.predict(new_df[features]) + np.log(new_df["cases_prev"] + 1)
        new_df["predicted_cases"] = round(np.maximum(np.exp(predictions) - 1, new_df["cases_prev"]))
        predictions = fatalities_model.predict(new_df[features]) + np.log(new_df["fat_prev"] + 1)
        new_df["predicted_fatalities"] = np.maximum(np.exp(predictions) - 1, new_df["fat_prev"])
        new_df["predicted_fatalities"] = round(np.minimum(new_df["predicted_fatalities"], new_df["predicted_cases"]*0.2))
        new_df.loc[new_df["Date"] == date, "ConfirmedCases"] = new_df.loc[new_df["Date"] == date, "predicted_cases"]
        new_df.loc[new_df["Date"] == date, "Fatalities"] = new_df.loc[new_df["Date"] == date, "predicted_fatalities"]
        pred_df = pred_df.append(new_df.loc[new_df["Date"] == date][pred_df.columns.tolist()])
            
        
            
        X_forecastId = sub_df.loc[(sub_df['key'] == key), ['ForecastId']]
        X_forecastId = X_forecastId.values.tolist()
        X_forecastId = [v[0] for v in X_forecastId]
          
for j in range(len(sub_df['ForecastId'])):
       
     dic = { 'ForecastId': X_forecastId[j], 'ConfirmedCases': pred_df['ConfirmedCases'][j], 'Fatalities': pred_df['Fatalities'][j]}
     sub.append(dic)

# %% [code]
submission = pd.DataFrame(sub)
submission[['ForecastId','ConfirmedCases','Fatalities']].to_csv(path_or_buf='submission.csv',index=False) '''
###########################################################################################################################################
sub = []
for date in scoring_dates.tolist():
    print(date)
    start=time.time()
    for key in key_list:
        if date < train_df['Date'].max():
            
            new_df = df.loc[df["Date"] < date].copy()
            curr_date_df = sub_df.loc[sub_df["Date"] == date].copy()
            curr_date_df["ConfirmedCases"] = 0
            curr_date_df["Fatalities"] = 0
            new_df = new_df.append(curr_date_df).reset_index(drop=True)
            new_df['cases_prev']=new_df['ConfirmedCases'].shift()
            new_df['fat_prev']=new_df['Fatalities'].shift() 
        
            predictions = cases_model.predict(new_df[features]) + np.log(new_df["cases_prev"] + 1)
            new_df["predicted_cases"] = round(np.maximum(np.exp(predictions) - 1, new_df["cases_prev"]))
            predictions = fatalities_model.predict(new_df[features]) + np.log(new_df["fat_prev"] + 1)
            new_df["predicted_fatalities"] = np.maximum(np.exp(predictions) - 1, new_df["fat_prev"])
            new_df["predicted_fatalities"] = round(np.minimum(new_df["predicted_fatalities"], new_df["predicted_cases"]*0.2))
            new_df.loc[new_df["Date"] == date, "ConfirmedCases"] = new_df.loc[new_df["Date"] == date, "predicted_cases"]
            new_df.loc[new_df["Date"] == date, "Fatalities"] = new_df.loc[new_df["Date"] == date, "predicted_fatalities"]
            pred_df = pred_df.append(new_df.loc[new_df["Date"] == date][pred_df.columns.tolist()])
            
        
       
        else:
            #new_df = new_df.copy()
            curr_date_df = sub_df.loc[sub_df["Date"] == date].copy()
            curr_date_df["ConfirmedCases"] = 0
            curr_date_df["Fatalities"] = 0
            new_df = new_df.append(curr_date_df).reset_index(drop=True)
            new_df['cases_prev']=new_df['ConfirmedCases'].shift()
            new_df['fat_prev']=new_df['Fatalities'].shift() 
        
            predictions = cases_model.predict(new_df[features]) + np.log(new_df["cases_prev"] + 1)
            new_df["predicted_cases"] = round(np.maximum(np.exp(predictions) - 1, new_df["cases_prev"]))
            predictions = fatalities_model.predict(new_df[features]) + np.log(new_df["fat_prev"] + 1)
            new_df["predicted_fatalities"] = np.maximum(np.exp(predictions) - 1, new_df["fat_prev"])
            new_df["predicted_fatalities"] = round(np.minimum(new_df["predicted_fatalities"], new_df["predicted_cases"]*0.2))
            new_df.loc[new_df["Date"] == date, "ConfirmedCases"] = new_df.loc[new_df["Date"] == date, "predicted_cases"]
            new_df.loc[new_df["Date"] == date, "Fatalities"] = new_df.loc[new_df["Date"] == date, "predicted_fatalities"]
            pred_df = pred_df.append(new_df.loc[new_df["Date"] == date][pred_df.columns.tolist()])
            
    print(time.time()-start)  
            
X_forecastId = sub_df.loc[(sub_df['key'] == key), ['ForecastId']]
X_forecastId = X_forecastId.values.tolist()
X_forecastId = [v[0] for v in X_forecastId]
            
                     
            
    
          
for j in range(len(sub_df['ForecastId'])):
          
    dic = { 'ForecastId': X_forecastId[j], 'ConfirmedCases': pred_df['ConfirmedCases'][j], 'Fatalities': pred_df['Fatalities'][j]}
    sub.append(dic)

# %% [code]
submission = pd.DataFrame(sub)
submission[['ForecastId','ConfirmedCases','Fatalities']].to_csv(path_or_buf='submission.csv',index=False) 
########################################################################################################################################################







In [ ]:
pred_df[pred_df['Country_Region']=='Italy']

In [ ]:
test_df['Date'].min()

In [ ]:
submission

In [ ]:
pred_df

In [ ]:
pred_df[pred_df["Country_Region"] == "Italy"]

In [ ]:
new_df['predicted_cases'][new_df["Country_Region"] == "Italy"]

In [ ]:
sub_df.info()

In [ ]:
test = pd.read_csv("../input/covid19-global-forecasting-week-4/test.csv")
test['Date']= pd.to_datetime(test['Date']) 
test = pd.merge(test, pred_df[["Province_State", "Country_Region", "Date", "ConfirmedCases", "Fatalities"]], on=["Province_State", "Country_Region", "Date"], how="left")


In [ ]:
new_df[new_df['Country_Region']=='Qatar']

In [ ]:
Predicted_data = pd.merge(test  ,
                 test_df[['ForecastId','Province_State','Country_Region','Date']],
                 on='ForecastId',how='left')

In [ ]:
df = pd.read_csv("../input/covid19-global-forecasting-week-4/train.csv")
df.columns

In [ ]:
merged_train_df.info()

In [ ]:
def get_nn_sub():
    df = pd.read_csv("../input/covid19-global-forecasting-week-4/train.csv")
    df['ConfirmedCases']=df['ConfirmedCases'].replace([-1], 0)
    df['Fatalities']=df['Fatalities'].replace([-1], 0)
    sub_df = pd.read_csv("../input/covid19-global-forecasting-week-4/test.csv")
    sub_df["key"]=sub_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)
    df["key"]=df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)
    features_columns = ['Id', 'Province_State', 'Country_Region', 'Date', 'ConfirmedCases',
       'Fatalities','mean_rate_case_last7', 'mean_rate_case_each7','cases_prev', 'fat_prev',
       'mean_rate_fat_last7', 'mean_rate_fat_each7', 'max_rate_case','mean_rate_case_last3','mean_rate_fat_last3',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat','Lat','Long',
       'max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities','total_pop', 'smokers_perc', 'density']

    '''coo_df = merged_train_df.groupby("Country_Region")[['mean_rate_case_last7', 'mean_rate_case_each7',
       'mean_rate_fat_last7', 'mean_rate_fat_each7', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat','cases_prev', 'fat_prev',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat','Lat','Long',
       'max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities','total_pop', 'smokers_perc', 'density']].mean().reset_index()'''
    
    coo_df = merged_train_df.groupby("Country_Region")[['mean_rate_case_last7', 'mean_rate_case_each7',
       'mean_rate_fat_last7', 'mean_rate_fat_each7', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case','mean_rate_case_last3','mean_rate_fat_last3',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat','Lat','Long','cases_prev', 'fat_prev',
       'max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities','total_pop', 'smokers_perc', 'density']].mean().reset_index()
   # coo_df = coo_df[coo_df["Country_Region"].notnull()]

    loc_group = ["Province_State", "Country_Region"]
    #loc_group = ["key"]
   # merged_train_df['Date']= pd.to_datetime(merged_train_df['Date']) 
  #  merged_train_df["days"] = (merged_train_df["Date"] - pd.to_datetime("2020-01-01")).dt.days

    def preprocess(df):
        df["Date"] = df["Date"].astype("datetime64[ms]")
  #  df["days"] = (df["Date"] - pd.to_datetime("2020-01-01")).dt.days
    #    df["weekend"] = df["Date"].dt.dayofweek//5

        df = df.merge(coo_df, how="left", on="Country_Region")
        #df = df.merge(coo_df, how="left", on="key")
        df["Lat"] = (df["Lat"] // 30).astype(np.float32).fillna(0)
        df["Long"] = (df["Long"] // 60).astype(np.float32).fillna(0)

        for col in loc_group:
            df[col].fillna("none", inplace=True)
        return df
    merged_train_df2=pd.DataFrame(columns=features_columns)
    merged_train_df2=merged_train_df[features_columns].copy()
    #df = merged_train_df
    df = preprocess(df)
    sub_df = preprocess(sub_df)
   # coo_df = coo_df[coo_df["Country_Region"].notnull()]

   # merged_train_df['Date']= pd.to_datetime(merged_train_df['Date']) 
  #  merged_train_df["days"] = (merged_train_df["Date"] - pd.to_datetime("2020-01-01")).dt.days


 

    print(df.shape)

    TARGETS = ["ConfirmedCases", "Fatalities"]

    for col in TARGETS:
        df[col] = np.log1p(df[col])

    NUM_SHIFT = 7
#'1-smokers_perc','density'
#2-'smokers_perc','density','mean_rate_case_last7'
#3-'smokers_perc','density','mean_rate_case_last7','mean_rate_fat_last7'
#4-'smokers_perc','density','mean_rate_fat_last7'
#5-'mean_rate_case_last7','mean_rate_fat_last7'
#6- Lat and Long
#7-'mean_rate_case_last7', 'mean_rate_fat_last7','std_rate_case','std_rate_fat','density','Lat','Long'
#8----'mean_rate_case_last7', 'mean_rate_fat_last7','std_rate_case', 'std_rate_fat','mean_rate_case', 'mean_rate_fat','rate_ConfirmedCases','rate_Fatalities'
#9-'std_rate_case','std_rate_fat','mean_rate_case', 'mean_rate_fat','rate_ConfirmedCases','rate_Fatalities'
#10--------'mean_rate_case_last7', 'mean_rate_case_each7',  'mean_rate_fat_last7', 'mean_rate_fat_each7',   'rate_ConfirmedCases', 'rate_Fatalities'
#11    'mean_rate_case_last7', 'mean_rate_case_each7','std_rate_case','std_rate_fat', 'max_to_min_rate_case', 'max_to_min_rate_fat',       'mean_rate_fat_last7', 'mean_rate_fat_each7'
#12-----'mean_rate_case_last7', 'mean_rate_case_each7','mean_rate_case_last3','mean_rate_fat_last3', 'max_to_min_rate_case','max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities', 'mean_rate_case', 'mean_rate_fat','std_rate_fat','std_rate_case',              'mean_rate_fat_last7', 'mean_rate_fat_each7   
    # we have to tune
    
    features = ['mean_rate_case_last7', 'mean_rate_case_each7','mean_rate_case_last3','mean_rate_fat_last3', 'max_to_min_rate_case',
                'max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities',
                'mean_rate_fat_last7', 'mean_rate_fat_each7']
  

    for s in range(1, NUM_SHIFT+1):
        for col in TARGETS:
            df["prev_{}_{}".format(col, s)] = df.groupby(loc_group)[col].shift(s)
            features.append("prev_{}_{}".format(col, s))

    df = df[df["Date"] >= df["Date"].min() + timedelta(days=NUM_SHIFT)].copy()

    TEST_FIRST = sub_df["Date"].min() # pd.to_datetime("2020-03-13") #
    TEST_DAYS = (df["Date"].max() - TEST_FIRST).days + 1

    dev_df, test_df = df[df["Date"] < TEST_FIRST].copy(), df[df["Date"] >= TEST_FIRST].copy()

    def nn_block(input_layer, size, dropout_rate, activation):
        out_layer = KL.Dense(size, activation=None)(input_layer)
        #out_layer = KL.BatchNormalization()(out_layer)
        out_layer = KL.Activation(activation)(out_layer)
        out_layer = KL.Dropout(dropout_rate)(out_layer)
        return out_layer
    '''def get_model():
        inp = KL.Input(shape=(len(features),))

        hidden_layer = nn_block(inp, 256,0, "relu")
        hidden_layer = nn_block(hidden_layer,128, 0.0, "relu")
        gate_layer = nn_block(hidden_layer,64, 0.0, "sigmoid")
        hidden_layer = nn_block(hidden_layer,128, 0, "relu")
        hidden_layer = nn_block(hidden_layer,64, 0.1, "relu") #0.05
        hidden_layer = KL.multiply([hidden_layer, gate_layer])

        out = KL.Dense(len(TARGETS), activation="linear")(hidden_layer)

        model = tf.keras.models.Model(inputs=[inp], outputs=out)
        return model'''
    '''def get_model():
        inp = KL.Input(shape=(len(features),))

        hidden_layer = nn_block(inp, 128,0, "relu")
        hidden_layer = nn_block(hidden_layer,64, 0.0, "relu")
        gate_layer = nn_block(hidden_layer,32, 0.0, "sigmoid")

        hidden_layer = nn_block(hidden_layer,32, 0.2, "relu") #0.05
        hidden_layer = KL.multiply([hidden_layer, gate_layer])

        out = KL.Dense(len(TARGETS), activation="linear")(hidden_layer)

        model = tf.keras.models.Model(inputs=[inp], outputs=out)
        return model'''


    def get_model():
        inp = KL.Input(shape=(len(features),))

        hidden_layer = nn_block(inp, 256, 0.0, "relu")
        gate_layer = nn_block(hidden_layer, 64, 0.0, "sigmoid")
        hidden_layer = nn_block(hidden_layer, 64, 0.1, "relu")
        hidden_layer = KL.multiply([hidden_layer, gate_layer])

        out = KL.Dense(len(TARGETS), activation="linear")(hidden_layer)

        model = tf.keras.models.Model(inputs=[inp], outputs=out)
        return model
    '''def get_model():
        inp = KL.Input(shape=(len(features),))

        hidden_layer = nn_block(inp, 64, 0.0, "relu")#64
        gate_layer = nn_block(hidden_layer, 32, 0.0, "sigmoid")#32
        hidden_layer = nn_block(hidden_layer, 32, 0, "relu")
        hidden_layer = KL.multiply([hidden_layer, gate_layer])

        out = KL.Dense(len(TARGETS), activation="linear")(hidden_layer)

        model = tf.keras.models.Model(inputs=[inp], outputs=out)
        return model'''

    get_model().summary()

    def get_input(df):
        return [df[features]]

    NUM_MODELS = 10


    def train_models(df, save=False):
        models = []
        for i in range(NUM_MODELS):
            model = get_model()
            model.compile(loss="mean_squared_error", optimizer=Nadam(lr=1e-4))
            hist = model.fit(get_input(df), df[TARGETS],
                             batch_size=2048, epochs=500, verbose=0, shuffle=True)
            if save:
                model.save_weights("model{}.h5".format(i))
            models.append(model)
        return models

    models = train_models(dev_df)


    prev_targets = ['prev_ConfirmedCases_1', 'prev_Fatalities_1']

    def predict_one(df, models):
        pred = np.zeros((df.shape[0], 2))
        for model in models:
            pred += model.predict(get_input(df))/len(models)
        pred = np.maximum(pred, df[prev_targets].values)
        pred[:, 0] = np.log1p(np.expm1(pred[:, 0]) + 0.1)
        pred[:, 1] = np.log1p(np.expm1(pred[:, 1]) + 0.01)
        return np.clip(pred, None, 15)

    print([mean_squared_error(dev_df[TARGETS[i]], predict_one(dev_df, models)[:, i]) for i in range(len(TARGETS))])


    def rmse(y_true, y_pred):
        return np.sqrt(mean_squared_error(y_true, y_pred))

    def evaluate(df):
        error = 0
        for col in TARGETS:
            error += rmse(df[col].values, df["pred_{}".format(col)].values)
        return np.round(error/len(TARGETS), 5)


    def predict(test_df, first_day, num_days, models, val=False):
        temp_df = test_df.loc[test_df["Date"] == first_day].copy()
        y_pred = predict_one(temp_df, models)

        for i, col in enumerate(TARGETS):
            test_df["pred_{}".format(col)] = 0
            test_df.loc[test_df["Date"] == first_day, "pred_{}".format(col)] = y_pred[:, i]

        print(first_day, np.isnan(y_pred).sum(), y_pred.min(), y_pred.max())
        if val:
            print(evaluate(test_df[test_df["Date"] == first_day]))


        y_prevs = [None]*NUM_SHIFT

        for i in range(1, NUM_SHIFT):
            y_prevs[i] = temp_df[['prev_ConfirmedCases_{}'.format(i), 'prev_Fatalities_{}'.format(i)]].values

        for d in range(1, num_days):
            date = first_day + timedelta(days=d)
            print(date, np.isnan(y_pred).sum(), y_pred.min(), y_pred.max())

            temp_df = test_df.loc[test_df["Date"] == date].copy()
            temp_df[prev_targets] = y_pred
            for i in range(2, NUM_SHIFT+1):
                temp_df[['prev_ConfirmedCases_{}'.format(i), 'prev_Fatalities_{}'.format(i)]] = y_prevs[i-1]

            y_pred, y_prevs = predict_one(temp_df, models), [None, y_pred] + y_prevs[1:-1]


            for i, col in enumerate(TARGETS):
                test_df.loc[test_df["Date"] == date, "pred_{}".format(col)] = y_pred[:, i]

            if val:
                print(evaluate(test_df[test_df["Date"] == date]))

        return test_df

    test_df = predict(test_df, TEST_FIRST, TEST_DAYS, models, val=True)
    print(evaluate(test_df))

    for col in TARGETS:
        test_df[col] = np.expm1(test_df[col])
        test_df["pred_{}".format(col)] = np.expm1(test_df["pred_{}".format(col)])

    models = train_models(df, save=True)

    sub_df_public = sub_df[sub_df["Date"] <= df["Date"].max()].copy()
    sub_df_private = sub_df[sub_df["Date"] > df["Date"].max()].copy()

    pred_cols = ["pred_{}".format(col) for col in TARGETS]
    #sub_df_public = sub_df_public.merge(test_df[["Date"] + loc_group + pred_cols].rename(columns={col: col[5:] for col in pred_cols}), 
    #                                    how="left", on=["Date"] + loc_group)
    sub_df_public = sub_df_public.merge(test_df[["Date"] + loc_group + TARGETS], how="left", on=["Date"] + loc_group)

    SUB_FIRST = sub_df_private["Date"].min()
    SUB_DAYS = (sub_df_private["Date"].max() - sub_df_private["Date"].min()).days + 1

    sub_df_private = df.append(sub_df_private, sort=False)

    for s in range(1, NUM_SHIFT+1):
        for col in TARGETS:
            sub_df_private["prev_{}_{}".format(col, s)] = sub_df_private.groupby(loc_group)[col].shift(s)

    sub_df_private = sub_df_private[sub_df_private["Date"] >= SUB_FIRST].copy()

    sub_df_private = predict(sub_df_private, SUB_FIRST, SUB_DAYS, models)

    for col in TARGETS:
        sub_df_private[col] = np.expm1(sub_df_private["pred_{}".format(col)])

    sub_df = sub_df_public.append(sub_df_private, sort=False)
    sub_df["ForecastId"] = sub_df["ForecastId"].astype(np.int16)

    return sub_df[["ForecastId"] + TARGETS]

In [ ]:
sub1 = get_cpmp_sub()
sub1['ForecastId'] = sub1['ForecastId'].astype('int')

In [ ]:
sub2 = get_nn_sub()

In [ ]:
sub1.sort_values("ForecastId", inplace=True)
sub2.sort_values("ForecastId", inplace=True)

In [ ]:
from sklearn.metrics import mean_squared_error

TARGETS = ["ConfirmedCases", "Fatalities"]

[np.sqrt(mean_squared_error(np.log1p(sub1[t].values), np.log1p(sub2[t].values))) for t in TARGETS]

In [ ]:
sub_df = sub1.copy()
for t in TARGETS:
    sub_df[t] = np.expm1(np.log1p(sub1[t].values)*0.00+ np.log1p(sub2[t].values)*1).astype('int')
    
sub_df.to_csv("submission.csv", index=False)

In [ ]:
Predicted_data = pd.merge(sub_df ,
                 test_df[['ForecastId','Province_State','Country_Region','Date']],
                 on='ForecastId',how='left')

In [ ]:
target1="ConfirmedCases"
target2="Fatalities"
key="Country_Region"
def trend(frame, key, target, new_target_name="trend"):
   
    corrections = 0
    group_keys = frame[ key].values.tolist()
    target = frame[target].values.tolist()
    trend=[1.0 for k in range (len(target))]

    for i in range(1, len(group_keys) - 1):
        previous_group = group_keys[i - 1]
        current_group = group_keys[i]

        previous_value = target[i - 1]
        current_value = target[i]
         
        if current_group == previous_group:
                if previous_value!=0.0:
                     trend[i]=current_value-previous_value

                 
        trend[i] =max(1,trend[i] )#correct negative values

    frame[new_target_name] = np.array(trend)
trend(Predicted_data, key, target1, new_target_name="trend_" +target1)
trend(Predicted_data, key, target2, new_target_name="trend_" +target2)    

In [ ]:
Predicted_data[Predicted_data['Country_Region']=='Qatar']


In [ ]:
confirmed_total_date = Predicted_data.groupby(['Date']).agg({'trend_ConfirmedCases':'sum','ConfirmedCases':'sum'})
fatalities_total_date = Predicted_data.groupby(['Date']).agg({'trend_Fatalities':'sum','Fatalities':'sum'})
print(confirmed_total_date)
print(fatalities_total_date)

In [ ]:
confirmed_date = train_df.groupby(['Date']).agg({'ConfirmedCases':['sum']})
fatalities_date = train_df.groupby(['Date']).agg({'Fatalities':['sum']})
print(confirmed_date)
print(fatalities_date)

In [ ]:
train_df[train_df['Country_Region']=='Qatar'].tail(10)

In [ ]:
def countryplot(train_df,country):
    
    confirmed_total = train_df[train_df['Country_Region']==country_dict[country]].groupby(['Date']).agg({'ConfirmedCases':['sum']})
    fatalities_total = train_df[train_df['Country_Region']==country_dict[country]].groupby(['Date']).agg({'Fatalities':['sum']})
    total = confirmed_total.join(fatalities_total)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(17,7))
    total.plot(ax=ax1)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=90)
    ax1.set_title(country + " Forecast Confirmed Cases and Fatalities ", size=13,)
    ax1.set_ylabel("Number of cases", size=13)
    ax1.set_xlabel("Date", size=13)
    #ax1.set_xticklabels("Date",rotation= 90)
    #plt.xticks(rotation=90)
    ax1.grid()
    fatalities_total.plot(ax=ax2, color='red')
    plt.grid()
    ax2.set_title(country+" Forecast Fatalities", size=13)
    ax2.set_ylabel("Number of cases", size=13)
    ax2.set_xlabel("Date", size=13)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
countrylist=['Qatar','Italy','Spain','United Arab Emirates','Egypt','Germany','France','Korea, South','Turkey','US','Saudi Arabia','United Kingdom']
for co in countrylist:
    countryplot(Predicted_data,co)
    #country='Italy'


In [ ]:
def countryplottotal(train_df):
      
    confirmed_total = train_df.groupby(['Date']).agg({'ConfirmedCases':['sum']})
    fatalities_total = train_df.groupby(['Date']).agg({'Fatalities':['sum']})
    total = confirmed_total.join(fatalities_total)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(17,7))
    total.plot(ax=ax1)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=90)
    ax1.set_title("World Wide Forecast Confirmed Cases and Fatalities ", size=13,)
    ax1.set_ylabel("Number of cases", size=13)
    ax1.set_xlabel("Date", size=13)
    #ax1.set_xticklabels("Date",rotation= 90)
    #plt.xticks(rotation=90)
    ax1.grid()
    fatalities_total.plot(ax=ax2, color='red')
    plt.grid()
    ax2.set_title("World Wide Forecast Fatalities", size=13)
    ax2.set_ylabel("Number of cases", size=13)
    ax2.set_xlabel("Date", size=13)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
countryplottotal(Predicted_data)

In [ ]:
def countryplothist(train_df,country):
    
    confirmed_total = train_df[train_df['Country_Region']==country_dict[country]].groupby(['Date']).agg({'trend_ConfirmedCases':['sum']})
    fatalities_total = train_df[train_df['Country_Region']==country_dict[country]].groupby(['Date']).agg({'trend_Fatalities':['sum']})
    total = confirmed_total.join(fatalities_total)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(17,7))
    total.plot(kind='bar',ax=ax1)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=90)
    ax1.set_title(country + " Forecast Confirmed Cases and Fatalities ", size=13,)
    ax1.set_ylabel("Number of cases", size=13)
    ax1.set_xlabel("Date", size=13)
    #ax1.set_xticklabels("Date",rotation= 90)
    #plt.xticks(rotation=90)
    ax1.grid()
    fatalities_total.plot(kind='bar',ax=ax2, color='red')
    plt.grid()
    ax2.set_title(country+" Forecast Fatalities", size=13)
    ax2.set_ylabel("Number of cases", size=13)
    ax2.set_xlabel("Date", size=13)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
countrylist=['Qatar','Italy','Spain','United Arab Emirates','Egypt','Germany','France','Korea, South','Turkey','US','Saudi Arabia','United Kingdom']
for co in countrylist:
    countryplothist(Predicted_data,co)

In [ ]:
train_df = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/train.csv")
test_df = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/test.csv")
submission = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/submission.csv")

train_df['ConfirmedCases']=train_df['ConfirmedCases'].replace([-1], 0)
train_df['Fatalities']=train_df['Fatalities'].replace([-1], 0)

train_df["key"]=train_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)
#train_df['cases_prev']=train_df.groupby("key")["ConfirmedCases"].shift()
#train_df['fat_prev']=train_df.groupby("key")["Fatalities"].shift()

train_df['cases_prev'] = train_df.groupby("key")["ConfirmedCases"].shift()
train_df['fat_prev'] = train_df.groupby("key")["Fatalities"].shift()

sub_df = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/test.csv")

sub_df["key"]=sub_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)

coo_d = merged_train_df.groupby('key')[['mean_rate_case_last7', 'mean_rate_case_each7',
       'mean_rate_fat_last7', 'mean_rate_fat_each7', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case','mean_rate_case_last3','mean_rate_fat_last3',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat','Lat','Long',
       'max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities','total_pop', 'smokers_perc', 'density']].mean().reset_index()
loc_group = ['key']
def preprocess(df):
        df["Date"] = df["Date"].astype("datetime64[ms]")
     #   df["days"] = (df["Date"] - pd.to_datetime("2020-01-01")).dt.days
    #    df["weekend"] = df["Date"].dt.dayofweek//5

        df = df.merge(coo_d, how="left", on='key')
        df["Lat"] = (df["Lat"] // 30).astype(np.float32).fillna(0)
        df["Long"] = (df["Long"] // 60).astype(np.float32).fillna(0)
        key_dummies = pd.get_dummies(df['key'])
        df = pd.concat([df,key_dummies],axis=1)
        for col in loc_group:
            df[col].fillna("none", inplace=True)
        return df


#x_train = merged_train_df2
sub_df=preprocess(sub_df)
x_train = preprocess(train_df)
x_test = sub_df
x_train.loc[x_train["Date"]<x_test['Date'].min(),"split"] = "train"
x_train.loc[x_train["Date"]>=x_test['Date'].min(),"split"] = "test"       

featureslist=x_train.columns.unique().tolist()
features=['mean_rate_case_last7', 'mean_rate_case_each7',
       'mean_rate_fat_last7', 'mean_rate_fat_each7', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case','Alabama_US',
 'Alaska_US',
 'Alberta_Canada',
 'Anguilla_United Kingdom',
 'Anhui_China',
 'Arizona_US',
 'Arkansas_US',
 'Aruba_Netherlands',
 'Australian Capital Territory_Australia',
 'Beijing_China',
 'Bermuda_United Kingdom',
 'Bonaire, Sint Eustatius and Saba_Netherlands',
 'British Columbia_Canada',
 'British Virgin Islands_United Kingdom',
 'California_US',
 'Cayman Islands_United Kingdom',
 'Channel Islands_United Kingdom',
 'Chongqing_China',
 'Colorado_US',
 'Connecticut_US',
 'Curacao_Netherlands',
 'Delaware_US',
 'District of Columbia_US',
 'Falkland Islands (Malvinas)_United Kingdom',
 'Faroe Islands_Denmark',
 'Florida_US',
 'French Guiana_France',
 'French Polynesia_France',
 'Fujian_China',
 'Gansu_China',
 'Georgia_US',
 'Gibraltar_United Kingdom',
 'Greenland_Denmark',
 'Guadeloupe_France',
 'Guam_US',
 'Guangdong_China',
 'Guangxi_China',
 'Guizhou_China',
 'Hainan_China',
 'Hawaii_US',
 'Hebei_China',
 'Heilongjiang_China',
 'Henan_China',
 'Hong Kong_China',
 'Hubei_China',
 'Hunan_China',
 'Idaho_US',
 'Illinois_US',
 'Indiana_US',
 'Inner Mongolia_China',
 'Iowa_US',
 'Isle of Man_United Kingdom',
 'Jiangsu_China',
 'Jiangxi_China',
 'Jilin_China',
 'Kansas_US',
 'Kentucky_US',
 'Liaoning_China',
 'Louisiana_US',
 'Macau_China',
 'Maine_US',
 'Manitoba_Canada',
 'Martinique_France',
 'Maryland_US',
 'Massachusetts_US',
 'Mayotte_France',
 'Michigan_US',
 'Minnesota_US',
 'Mississippi_US',
 'Missouri_US',
 'Montana_US',
 'Montserrat_United Kingdom',
 'Nebraska_US',
 'Nevada_US',
 'New Brunswick_Canada',
 'New Caledonia_France',
 'New Hampshire_US',
 'New Jersey_US',
 'New Mexico_US',
 'New South Wales_Australia',
 'New York_US',
 'Newfoundland and Labrador_Canada',
 'Ningxia_China',
 'North Carolina_US',
 'North Dakota_US',
 'Northern Territory_Australia',
 'Northwest Territories_Canada',
 'Nova Scotia_Canada',
 'Ohio_US',
 'Oklahoma_US',
 'Ontario_Canada',
 'Oregon_US',
 'Pennsylvania_US',
 'Prince Edward Island_Canada',
 'Puerto Rico_US',
 'Qinghai_China',
 'Quebec_Canada',
 'Queensland_Australia',
 'Reunion_France',
 'Rhode Island_US',
 'Saint Barthelemy_France',
 'Saint Pierre and Miquelon_France',
 'Saskatchewan_Canada',
 'Shaanxi_China',
 'Shandong_China',
 'Shanghai_China',
 'Shanxi_China',
 'Sichuan_China',
 'Sint Maarten_Netherlands',
 'South Australia_Australia',
 'South Carolina_US',
 'South Dakota_US',
 'St Martin_France',
 'Tasmania_Australia',
 'Tennessee_US',
 'Texas_US',
 'Tianjin_China',
 'Tibet_China',
 'Turks and Caicos Islands_United Kingdom',
 'Utah_US',
 'Vermont_US',
 'Victoria_Australia',
 'Virgin Islands_US',
 'Virginia_US',
 'Washington_US',
 'West Virginia_US',
 'Western Australia_Australia',
 'Wisconsin_US',
 'Wyoming_US',
 'Xinjiang_China',
 'Yukon_Canada',
 'Yunnan_China',
 'Zhejiang_China',
 'nan_Afghanistan',
 'nan_Albania',
 'nan_Algeria',
 'nan_Andorra',
 'nan_Angola',
 'nan_Antigua and Barbuda',
 'nan_Argentina',
 'nan_Armenia',
 'nan_Austria',
 'nan_Azerbaijan',
 'nan_Bahamas',
 'nan_Bahrain',
 'nan_Bangladesh',
 'nan_Barbados',
 'nan_Belarus',
 'nan_Belgium',
 'nan_Belize',
 'nan_Benin',
 'nan_Bhutan',
 'nan_Bolivia',
 'nan_Bosnia and Herzegovina',
 'nan_Botswana',
 'nan_Brazil',
 'nan_Brunei',
 'nan_Bulgaria',
 'nan_Burkina Faso',
 'nan_Burma',
 'nan_Burundi',
 'nan_Cabo Verde',
 'nan_Cambodia',
 'nan_Cameroon',
 'nan_Central African Republic',
 'nan_Chad',
 'nan_Chile',
 'nan_Colombia',
 'nan_Congo (Brazzaville)',
 'nan_Congo (Kinshasa)',
 'nan_Costa Rica',
 "nan_Cote d'Ivoire",
 'nan_Croatia',
 'nan_Cuba',
 'nan_Cyprus',
 'nan_Czechia',
 'nan_Denmark',
 'nan_Diamond Princess',
 'nan_Djibouti',
 'nan_Dominica',
 'nan_Dominican Republic',
 'nan_Ecuador',
 'nan_Egypt',
 'nan_El Salvador',
 'nan_Equatorial Guinea',
 'nan_Eritrea',
 'nan_Estonia',
 'nan_Eswatini',
 'nan_Ethiopia',
 'nan_Fiji',
 'nan_Finland',
 'nan_France',
 'nan_Gabon',
 'nan_Gambia',
 'nan_Georgia',
 'nan_Germany',
 'nan_Ghana',
 'nan_Greece',
 'nan_Grenada',
 'nan_Guatemala',
 'nan_Guinea',
 'nan_Guinea-Bissau',
 'nan_Guyana',
 'nan_Haiti',
 'nan_Holy See',
 'nan_Honduras',
 'nan_Hungary',
 'nan_Iceland',
 'nan_India',
 'nan_Indonesia',
 'nan_Iran',
 'nan_Iraq',
 'nan_Ireland',
 'nan_Israel',
 'nan_Italy',
 'nan_Jamaica',
 'nan_Japan',
 'nan_Jordan',
 'nan_Kazakhstan',
 'nan_Kenya',
 'nan_Korea, South',
 'nan_Kosovo',
 'nan_Kuwait',
 'nan_Kyrgyzstan',
 'nan_Laos',
 'nan_Latvia',
 'nan_Lebanon',
 'nan_Liberia',
 'nan_Libya',
 'nan_Liechtenstein',
 'nan_Lithuania',
 'nan_Luxembourg',
 'nan_MS Zaandam',
 'nan_Madagascar',
 'nan_Malawi',
 'nan_Malaysia',
 'nan_Maldives',
 'nan_Mali',
 'nan_Malta',
 'nan_Mauritania',
 'nan_Mauritius',
 'nan_Mexico',
 'nan_Moldova',
 'nan_Monaco',
 'nan_Mongolia',
 'nan_Montenegro',
 'nan_Morocco',
 'nan_Mozambique',
 'nan_Namibia',
 'nan_Nepal',
 'nan_Netherlands',
 'nan_New Zealand',
 'nan_Nicaragua',
 'nan_Niger',
 'nan_Nigeria',
 'nan_North Macedonia',
 'nan_Norway',
 'nan_Oman',
 'nan_Pakistan',
 'nan_Panama',
 'nan_Papua New Guinea',
 'nan_Paraguay',
 'nan_Peru',
 'nan_Philippines',
 'nan_Poland',
 'nan_Portugal',
 'nan_Qatar',
 'nan_Romania',
 'nan_Russia',
 'nan_Rwanda',
 'nan_Saint Kitts and Nevis',
 'nan_Saint Lucia',
 'nan_Saint Vincent and the Grenadines',
 'nan_San Marino',
 'nan_Sao Tome and Principe',
 'nan_Saudi Arabia',
 'nan_Senegal',
 'nan_Serbia',
 'nan_Seychelles',
 'nan_Sierra Leone',
 'nan_Singapore',
 'nan_Slovakia',
 'nan_Slovenia',
 'nan_Somalia',
 'nan_South Africa',
 'nan_South Sudan',
 'nan_Spain',
 'nan_Sri Lanka',
 'nan_Sudan',
 'nan_Suriname',
 'nan_Sweden',
 'nan_Switzerland',
 'nan_Syria',
 'nan_Taiwan*',
 'nan_Tanzania',
 'nan_Thailand',
 'nan_Timor-Leste',
 'nan_Togo',
 'nan_Trinidad and Tobago',
 'nan_Tunisia',
 'nan_Turkey',
 'nan_Uganda',
 'nan_Ukraine',
 'nan_United Arab Emirates',
 'nan_United Kingdom',
 'nan_Uruguay',
 'nan_Uzbekistan',
 'nan_Venezuela',
 'nan_Vietnam',
 'nan_West Bank and Gaza',
 'nan_Western Sahara',
 'nan_Zambia',
 'nan_Zimbabwe',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat','mean_rate_case_last3','mean_rate_fat_last3',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat','Lat','Long',
       'max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities','total_pop', 'smokers_perc', 'density',]
def train_model(df, label, base_label, features=features, **kwargs):
    X_train = df.loc[df["split"] == "train"][features]
    y_train = np.log(df.loc[df["split"] == "train"][label] + 1)
    b_train = np.log(df.loc[df["split"] == "train"][base_label] + 1)
    X_test = df.loc[df["split"] == "test"][features]
    y_test = np.log(df.loc[df["split"] == "test", label] + 1)
    b_test = np.log(df.loc[df["split"] == "test", base_label] + 1)
    print(kwargs)
    model = lgb.LGBMRegressor(**kwargs)
    model.fit(X_train, y_train, init_score = b_train)
    y_pred = model.predict(X_test)
    print(np.sqrt(mean_squared_error(y_test, y_pred + b_test)))
   # print(len(X_test))
    return model







In [ ]:
lgb_model_cases = train_model(x_train,"ConfirmedCases",'cases_prev',features,
                                   max_depth=5,
                                   colsample_bytree=0.8,
                                   learning_rate=0.1,
                                   n_estimators=1000,
                                   subsample=0.8)

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(lgb_model_cases.feature_importances_,x_train[features2])), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-01.png')



lgb_model_fatalities = train_model(x_train, "Fatalities",'fat_prev',features2, 
                                   max_depth=5,
                                   colsample_bytree=0.8,
                                   learning_rate=0.1,
                                   n_estimators=500,
                                   subsample=0.8
                                  )




In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(lgb_model_fatalities.feature_importances_,x_train[features2])), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-01.png')






In [ ]:
X_train = x_train[features2]
y_train = np.log(x_train["ConfirmedCases"] + 1)
b_train = np.log(x_train["cases_prev"] + 1)
cases_model = lgb.LGBMRegressor(max_depth=5,
                                   colsample_bytree=0.8,
                                   learning_rate=0.1,
                                   n_estimators=500,
                                   subsample=0.8
                               )
cases_model.fit(X_train, y_train, init_score = b_train)

X_train = x_train[features2]
y_train = np.log(x_train["Fatalities"] + 1)
b_train = np.log(x_train["fat_prev"] + 1)
fatalities_model = lgb.LGBMRegressor(max_depth=5,
                                   colsample_bytree=0.8,
                                   learning_rate=0.1,
                                   n_estimators=500,
                                   subsample=0.8)
fatalities_model.fit(X_train, y_train, init_score = b_train)

In [ ]:
df = pd.read_csv("train.csv")
sub_df = pd.read_csv("test.csv")
df["key"]=train_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)
sub_df["key"]=sub_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)


coo_df = merged_train_df.groupby("key")[['mean_rate_case_last7', 'mean_rate_case_each7',
       'mean_rate_fat_last7', 'mean_rate_fat_each7', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat','mean_rate_case_last3','mean_rate_fat_last3',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat','Lat','Long',
       'max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities','total_pop', 'smokers_perc', 'density']].mean().reset_index()
   # coo_df = coo_df[coo_df["Country_Region"].notnull()]

loc_group = ["key"]
   # merged_train_df['Date']= pd.to_datetime(merged_train_df['Date']) 
  #  merged_train_df["days"] = (merged_train_df["Date"] - pd.to_datetime("2020-01-01")).dt.days

def preprocess(df):
    df["Date"] = df["Date"].astype("datetime64[ms]")
  #  df["days"] = (df["Date"] - pd.to_datetime("2020-01-01")).dt.days
    #    df["weekend"] = df["Date"].dt.dayofweek//5

    df = df.merge(coo_df, how="left", on="key")
    df["Lat"] = (df["Lat"] // 30).astype(np.float32).fillna(0)
    df["Long"] = (df["Long"] // 60).astype(np.float32).fillna(0)
    key_dummies = pd.get_dummies(df['key'])
    df = pd.concat([df,key_dummies],axis=1)

    for col in loc_group:
        df[col].fillna("none", inplace=True)
    return df

df = preprocess(df)
sub_df = preprocess(sub_df)


train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

#train_df['Date']= pd.to_datetime(train_df['Date']) 
country_list = train_df ['Country_Region'].unique()
key_list=df['key'].unique()
#train_df['Date']= pd.to_datetime(train_df['Date']) 


scoring_dates = test_df['Date'].unique()
pred_df = pd.DataFrame(columns=train_df.columns)

sub = []
for date in scoring_dates.tolist():
    print(date)
    start=time.time()
    
    if date < train_df['Date'].max():
            
        new_df = df.loc[df["Date"] < date].copy()
        curr_date_df = sub_df.loc[sub_df["Date"] == date].copy()
        curr_date_df["ConfirmedCases"] = 0
        curr_date_df["Fatalities"] = 0
        new_df = new_df.append(curr_date_df).reset_index(drop=True)
        new_df['cases_prev']=new_df.groupby("key")["ConfirmedCases"].shift()
        new_df['fat_prev']=new_df.groupby("key")["Fatalities"].shift()
        
        
    

        predictions = cases_model.predict(new_df[features2]) + np.log(new_df["cases_prev"] + 1)
        new_df["predicted_cases"] = round(np.maximum(np.exp(predictions) - 1, new_df["cases_prev"]))
        predictions = fatalities_model.predict(new_df[features2]) + np.log(new_df["fat_prev"] + 1)
        new_df["predicted_fatalities"] = np.maximum(np.exp(predictions) - 1, new_df["fat_prev"])
        new_df["predicted_fatalities"] = round(np.minimum(new_df["predicted_fatalities"], new_df["predicted_cases"]*0.2))
        new_df.loc[new_df["Date"] == date, "ConfirmedCases"] = new_df.loc[new_df["Date"] == date, "predicted_cases"]
        new_df.loc[new_df["Date"] == date, "Fatalities"] = new_df.loc[new_df["Date"] == date, "predicted_fatalities"]
        pred_df = pred_df.append(new_df.loc[new_df["Date"] == date][pred_df.columns.tolist()])
            
        
       
    else:
            #new_df = new_df.copy()
        curr_date_df = sub_df.loc[sub_df["Date"] == date].copy()
        curr_date_df["ConfirmedCases"] = 0
        curr_date_df["Fatalities"] = 0
        new_df = new_df.append(curr_date_df).reset_index(drop=True)
        new_df['cases_prev']=new_df.groupby("key")["ConfirmedCases"].shift()
        new_df['fat_prev']=new_df.groupby("key")["Fatalities"].shift()
        
        predictions = cases_model.predict(new_df[features2]) + np.log(new_df["cases_prev"] + 1)
        new_df["predicted_cases"] = round(np.maximum(np.exp(predictions) - 1, new_df["cases_prev"]))
        predictions = fatalities_model.predict(new_df[features2]) + np.log(new_df["fat_prev"] + 1)
        new_df["predicted_fatalities"] = np.maximum(np.exp(predictions) - 1, new_df["fat_prev"])
        new_df["predicted_fatalities"] = round(np.minimum(new_df["predicted_fatalities"], new_df["predicted_cases"]*0.2))
        new_df.loc[new_df["Date"] == date, "ConfirmedCases"] = new_df.loc[new_df["Date"] == date, "predicted_cases"]
        new_df.loc[new_df["Date"] == date, "Fatalities"] = new_df.loc[new_df["Date"] == date, "predicted_fatalities"]
        pred_df = pred_df.append(new_df.loc[new_df["Date"] == date][pred_df.columns.tolist()])
            
    print(time.time()-start)  
            
X_forecastId = sub_df.loc[(sub_df['key'] == key), ['ForecastId']]
X_forecastId = X_forecastId.values.tolist()
X_forecastId = [v[0] for v in X_forecastId]
for j in range(len(sub_df['ForecastId'])):
          
    dic = { 'ForecastId': X_forecastId[j], 'ConfirmedCases': pred_df['ConfirmedCases'][j], 'Fatalities': pred_df['Fatalities'][j]}
    sub.append(dic)

submission = pd.DataFrame(sub)
submission[['ForecastId','ConfirmedCases','Fatalities']].to_csv(path_or_buf='submission.csv',index=False) 
from Gaussiandistribution import Gaussian